**OOV 문제**
: 모르는 단어때문에 문제 풀이가 제대로 안됨 </br>
**단어 분리(Subword segmenation)**
하나의 단어는 여러 내부 단어들의 조합, 단어를 여러 개로 분리 후 단어를 이해하겠다는 전처리 작업 </br>
=> 기계가 oov에 대해 어느 정도 대처할 수 있게 하며 주요 전처리로 사용됨

# BPE :: Byte Pair Encoding
=> 대표적 단어 분리 토크나이저 </br> 
참조 :: https://wikidocs.net/22592 </br>
1994년 나온 데이터 압축 알고리즘, 그 후 자연어처리의 단어 분리 알고리즘으로 응용됨 </br>
=> 글자(character)단위에서 점점 단어 집합을 만들어내는 Bottom up 방식의 전개 </br>
: 단어들을 글자 또는 유니코드 단위로 단어 집합을 만든 후 가장 많이 증장하는 유니그램을 하나의 유니그램으로 통합


In [0]:
import re, collections

In [0]:
epochs=10

In [0]:
vocab = {'l o w </w>' :5, #</w> : 단어의 맨 끝에 사용, 각 단어를 글자단위 분리
         'l o w e r </w>' :2,
         'n e w e s t</w>':6,
         'w i d e s t</w>':3}

In [0]:
def get_stats(vocab):
  pairs = collections.defaultdict(int)
  for word, freq in vocab.items():
    symbols = word.split()
    for i in range(len(symbols)-1):
      pairs[symbols[i], symbols[i+1]] += freq
  return pairs

In [22]:
get_stats(vocab)

defaultdict(int,
            {('e', 'r'): 2,
             ('low', 'e'): 2,
             ('r', '</w>'): 2,
             ('wid', 'est</w>'): 3})

In [0]:
def merge_vocab(pair, v_in):
  v_out = {}
  # re.escape : 영문자, 숫자가 아닌 문자에 대해 백슬러시 추가
  # 메타문자(특수 문자 기호) 포함 문자열을 정규식 변경 가능
  bigram = re.escape(' '.join(pair))
  p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
  for word in v_in:
    w_out = p.sub(''.join(pair), word)
    v_out[w_out] = v_in[word]
  return v_out

In [21]:
for i in range(epochs):
  pairs = get_stats(vocab)
  best = max(pairs, key=pairs.get)
  vocab = merge_vocab(best, vocab)
  print(best)

('e', 's')
('es', 't</w>')
('l', 'o')
('lo', 'w')
('n', 'e')
('ne', 'w')
('new', 'est</w>')
('low', '</w>')
('w', 'i')
('wi', 'd')


˙ 실무에서는 단어 분리를 위해 구글의 센텐스피스(sentencepiece) 사용 </br>
=> 사전 토큰화 작업 없이 전처리 적용 안 된 데이터에 바로 단어 분리 토큰화 수행, 언어에 종속되지 않는다